In [5]:
import re, hashlib
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image, features
import re, os, json, hashlib
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image, ImageOps, ImageFilter, ImageDraw, ImageFont
import numpy as np
from PIL import Image, features
from io import BytesIO
BASE = "https://www.kaitorishouten-co.jp"
LIST_URL = f"{BASE}/products/1/list_category/711"  # 换成当前分类页

BASE = "https://www.kaitorishouten-co.jp"
CAT_URL = f"{BASE}/category/1/711"   # ← 改用完整分类页
UA = "Mozilla/5.0"
CACHE_FILE = "sprite_map_cache.json"

def open_image_bytes(raw, ct):
    # 尝试 PNG / WEBP；若是 AVIF，请安装 pillow-heif 或 pillow-avif-plugin
    try:
        return Image.open(BytesIO(raw)).convert("RGBA")
    except Exception as e1:
        if ("avif" in (ct or "").lower()) or (raw[4:8]==b"ftyp"):
            try:
                from pillow_heif import register_heif_opener
                register_heif_opener()
                return Image.open(BytesIO(raw)).convert("RGBA")
            except Exception:
                import pillow_avif  # noqa: F401
                return Image.open(BytesIO(raw)).convert("RGBA")
        raise

def sniff_is_image(b):
    # 支持 PNG/WebP/AVIF/GIF 的魔数嗅探
    if b.startswith(b"\x89PNG\r\n\x1a\n"): return "png"
    if b.startswith(b"RIFF") and b[8:12] == b"WEBP": return "webp"
    if b[:12].find(b"ftyp") == 4: return "avif/iso-bmff"
    if b.startswith(b"GIF87a") or b.startswith(b"GIF89a"): return "gif"
    return ""

def extract_csrf_token(html: str) -> str | None:
    soup = BeautifulSoup(html, 'html.parser')
    # 1) 精确：<meta name="eccube-csrf-token" content="...">
    m = soup.find('meta', attrs={'name': lambda v: v and v.lower()=='eccube-csrf-token'})
    if m and m.get('content'):
        return m['content']
    # 2) 宽松：任何 name 含 csrf 的 meta
    m = soup.find('meta', attrs={'name': lambda v: v and 'csrf' in v.lower()})
    if m and m.get('content'):
        return m['content']
    # 3) 正则兜底（属性顺序/单双引号）
    r = re.search(r'name=["\']eccube-csrf-token["\'][^>]*content=["\']([^"\']+)["\']', html, re.I)
    if r: return r.group(1)
    r = re.search(r'content=["\']([^"\']+)["\'][^>]*name=["\']eccube-csrf-token["\']', html, re.I)
    if r: return r.group(1)
    return None

def extract_sprite_info(soup):
    """从 <style> 里抓 .encrypt-num 的 background-image/width/height"""
    css_texts = []
    for s in soup.find_all("style"):
        t = s.string or s.text or ""
        if ".encrypt-num" in t or "encrypt-num" in t:
            css_texts.append(t)
    css = "\n".join(css_texts)

    # 背景图
    m_url = re.search(r'\.encrypt-num[^}]*background-image\s*:\s*url\((["\']?)([^)"\']+)\1\)', css)
    sprite_url = m_url.group(2).strip() if m_url else None

    # 单格宽/高
    m_w = re.search(r'\.encrypt-num[^}]*width\s*:\s*(\d+)px', css)
    m_h = re.search(r'\.encrypt-num[^}]*height\s*:\s*(\d+)px', css)
    cell_w = int(m_w.group(1)) if m_w else 10
    cell_h = int(m_h.group(1)) if m_h else 16

    if sprite_url and sprite_url.startswith("//"):
        sprite_url = "https:" + sprite_url
    elif sprite_url and sprite_url.startswith("/"):
        sprite_url = urljoin(BASE, sprite_url)

    return sprite_url, cell_w, cell_h

def md5_bytes(b):
    return hashlib.md5(b).hexdigest()

def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_cache(d):
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(d, f, ensure_ascii=False, indent=2)

def cut_tiles(img: Image.Image, cell_w, cell_h):
    cols = img.width // cell_w
    rows = img.height // cell_h
    tiles = []
    for r in range(rows):
        for c in range(cols):
            box = (c*cell_w, r*cell_h, (c+1)*cell_w, (r+1)*cell_h)
            tiles.append(img.crop(box))
    return tiles, cols, rows

def upscale_and_binarize(tile: Image.Image, scale=4):
    """放大+阈值，便于 OCR"""
    big = tile.resize((tile.width*scale, tile.height*scale), Image.NEAREST)
    gray = ImageOps.grayscale(big)
    # 自动阈值（Otsu 简化）
    arr = np.array(gray)
    thr = int(arr.mean())
    bw = gray.point(lambda p: 255 if p > thr else 0, mode='1')
    return bw.convert("L")

def try_ocr_tiles(tiles):
    try:
        import pytesseract  # 若没装，会走 except
    except Exception:
        return None  # 触发半自动标注

    out = []
    for t in tiles:
        img = upscale_and_binarize(t, scale=5)
        # 只允许数字与逗号；psm 10 = treat the image as a single character
        txt = pytesseract.image_to_string(
            img, config='--psm 10 -c tessedit_char_whitelist=0123456789,'
        )
        ch = re.sub(r'[^0-9,]', '', txt).strip()
        out.append(ch[:1] if ch else "")
    return out

def manual_label_tiles(tiles):
    """在控制台人工标注（一次 11 个），并生成一张标注预览 png"""
    grid = Image.new("RGB", (len(tiles)*tiles[0].width, tiles[0].height), "white")
    draw = ImageDraw.Draw(grid)
    x = 0
    for i, t in enumerate(tiles):
        grid.paste(t.convert("RGB"), (x, 0))
        # 标注格序号
        draw.text((x+1, 10), str(i), fill=(255,0,0))
        x += t.width
    grid.save("sprite_tiles_preview.png")
    print("已输出预览：sprite_tiles_preview.png  请参照图片从左到右输入每格字符（仅 0-9 与 ,）")
    labels = []
    for i in range(len(tiles)):
        while True:
            ch = input(f"第 {i} 格字符 = ").strip()
            if ch in list("0123456789,"):
                labels.append(ch)
                break
            else:
                print("只允许输入 0-9 或 ,")
    return labels

def build_idx_to_char(img, cell_w, cell_h):
    tiles, cols, rows = cut_tiles(img, cell_w, cell_h)
    # 很多站一行就是 11 格，若更多格，优先取前 12 格（0-9 与 , 的候选）
    max_use = min(16, len(tiles))
    tiles = tiles[:max_use]

    labels = try_ocr_tiles(tiles)
    if not labels or sum(1 for x in labels if x) < 8:
        # OCR 不足，转半自动
        labels = manual_label_tiles(tiles)

    # idx->char
    idx2char = {}
    for i, ch in enumerate(labels):
        if ch:
            idx2char[i] = ch

    # 逗号兜底：很多站逗号单独一格，OCR容易空。若 labels 不含逗号，尝试猜测
    if "," not in idx2char.values():
        # 经验：逗号 tile 通常“墨迹”最少，可用像素点计数找最稀疏的一格当 ,
        fg_counts = []
        for i, t in enumerate(tiles):
            arr = np.array(t.convert("L"))
            fg = (arr < 200).sum()  # 简单计数（黑/深灰像素）
            fg_counts.append((fg, i))
        # 选最小的那个作为逗号（如果未被填充）
        fg_counts.sort()
        for _, i in fg_counts:
            if i not in idx2char:
                idx2char[i] = ","
                break
    return idx2char

def collect_used_indices(soup, cell_w):
    """从页面上提取所有出现过的 x 偏移 -> 索引"""
    used = set()
    for sp in soup.select("span.encrypt-num"):
        st = sp.get("style","")
        m = re.search(r'background-position\s*:\s*(-?\d+)px', st) or re.search(r'(-?\d+)px', st)
        if m:
            x = int(m.group(1))
            idx = (-x) // cell_w
            used.add(idx)
    return sorted(used)

def decode_price(tr, cell_w, idx2char):
    digits = []
    for sp in tr.select("span.encrypt-num"):
        st = sp.get("style","")
        m = re.search(r'background-position\s*:\s*(-?\d+)px', st) or re.search(r'(-?\d+)px', st)
        if not m:
            digits.append("?"); continue
        x = int(m.group(1))
        idx = (-x) // cell_w
        ch = idx2char.get(idx, "?")
        digits.append(ch)
    return "".join(digits)





with requests.Session() as s:
    s.headers.update({
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    'Accept-Language': 'ja,en;q=0.9,zh-CN;q=0.8',
    })
    r0 = s.get('https://www.kaitorishouten-co.jp/')
    token = extract_csrf_token(r0.text)
    r1 = s.get(LIST_URL)
    token2 = extract_csrf_token(r1.text) or token
    headers = {
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'https://www.kaitorishouten-co.jp/category/1/709',
        "Accept": "image/webp,image/png,*/*;q=0.8",
    }
    if token2:
        headers['eccube-csrf-token'] = token2
    # 1) 抓完整分类页（拿到 cookie + DOM + <style>）
    r = s.get(CAT_URL,headers=headers, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.content, "html.parser")

    # 2) 从 <style> 里找到雪碧图 URL 和单格尺寸
    css = "\n".join((st.string or st.text or "") for st in soup.find_all("style"))
    m_url = re.search(r'\.encrypt-num[^}]*background-image\s*:\s*url\((["\']?)([^)\'"]+)\1\)', css)
    assert m_url, "没在页面样式里找到 .encrypt-num 的 background-image"
    sprite_url = urljoin(BASE, m_url.group(2))
    m_w = re.search(r'\.encrypt-num[^}]*width\s*:\s*(\d+)px', css)
    m_h = re.search(r'\.encrypt-num[^}]*height\s*:\s*(\d+)px', css)
    CELL_W = int(m_w.group(1)) if m_w else 10
    CELL_H = int(m_h.group(1)) if m_h else 16

    # 3) 抓雪碧图（同一会话 + Referer；拒绝 AVIF）
    rs = s.get(sprite_url, headers={
        "Referer": CAT_URL,
        "Accept": "image/webp,image/png,*/*;q=0.8",
    }, timeout=30)
    rs.raise_for_status()
    raw = rs.content
    ct = rs.headers.get("Content-Type","").lower()
    img = open_image_bytes(raw, ct)  # ← 这里就能打开
    print("sprite:", sprite_url, "|", ct, "| MD5:", hashlib.md5(raw).hexdigest())


    # 4) 判定是不是图片；若拿到 HTML，落盘调试
    raw = rs.content
    ct  = (rs.headers.get("Content-Type") or "").lower()
    print("HTTP", rs.status_code, "Content-Type:", ct)

    if (ct.startswith("text/html") or not ct) and (raw[:64].lstrip().startswith(b"<!") or b"<html" in raw[:256].lower()):
        open("sprite_debug.html","wb").write(raw)
        raise RuntimeError("拿到的是 HTML（可能反盗链/Referer 缺失/旧 token）。已保存 sprite_debug.html 供查看。")

    kind = sniff_is_image(raw) or ("image/* via header" if ct.startswith("image/") else "")
    if not (ct.startswith("image/") or kind):
        # 有些 CDN 会用 application/octet-stream 返回图片，这里放宽判定
        if ct.startswith("application/octet-stream") or kind:
            pass
        else:
            head = raw[:64]
            raise RuntimeError(f"响应看起来不是图片，Content-Type={ct!r} 头部={head!r}")

    # 5) 解码（包含 AVIF 兜底）
    def open_image_bytes(raw_bytes):
        try:
            return Image.open(BytesIO(raw_bytes)).convert("RGBA")
        except Exception as e1:
            # AVIF 处理：安装 pillow-heif 或 pillow-avif-plugin 任一
            try:
                from pillow_heif import register_heif_opener
                register_heif_opener()
                return Image.open(BytesIO(raw_bytes)).convert("RGBA")
            except Exception:
                try:
                    import pillow_avif  # noqa: F401
                    return Image.open(BytesIO(raw_bytes)).convert("RGBA")
                except Exception as e2:
                    raise RuntimeError(
                        f"无法用 Pillow 打开图片：{e1}\n"
                        "若服务器返回 AVIF，请 pip install pillow-heif 或 pillow-avif-plugin；"
                        "或把请求头 Accept 改为 'image/webp,image/png,*/*;q=0.8' 再试。"
                    )

    img = open_image_bytes(raw)
    print("Image size:", img.size, "mode:", img.mode, "MD5:", hashlib.md5(raw).hexdigest())
    img.show()

    idx2char = build_idx_to_char(img, CELL_W, CELL_H)
    rows = raw.select("tr.price_list_item")
    for tr in rows:
        price = decode_price(tr, CELL_W, idx2char)
        name = " ".join(tr.select_one("td:nth-of-type(2)").stripped_strings)[:60]
        print(price, "｜", name)



sprite: https://www.kaitorishouten-co.jp/products/encrypt_price/TEFkcTVod0wxYVQ2ZkE9PQ== | image/png | MD5: ee6231f25b30c4ca86760d18b0fb7bc2
HTTP 200 Content-Type: image/png
Image size: (110, 16) mode: RGBA MD5: ee6231f25b30c4ca86760d18b0fb7bc2
已输出预览：sprite_tiles_preview.png  请参照图片从左到右输入每格字符（仅 0-9 与 ,）


Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt


KeyboardInterrupt: Interrupted by user

In [8]:
try:
    import pytesseract as pt
except Exception as e:
    print("pytesseract 未安装：", e)
    print("请先:  pip install pytesseract")

print("pytesseract:", getattr(pt, "__version__", "unknown"))
try:
    print("tesseract 版本:", pt.get_tesseract_version())
except Exception as e:
    print("无法读取 tesseract 版本：", e, "\n可能未安装 tesseract 可执行文件。")
    print("macOS:  brew install tesseract\nUbuntu: sudo apt-get install tesseract-ocr\nWindows: 安装官方exe后把路径加到 PATH")
img = Image.new("RGB", (400, 120), "white")
draw = ImageDraw.Draw(img)
draw.text((10, 10), "0123456789, 123,456", fill="black")  # 用系统默认字体
img.save("tess_smoke_input.png")
print("已保存合成测试图 tess_smoke_input.png")

text = pt.image_to_string(img, config="--psm 6 -c tessedit_char_whitelist=0123456789,")
print("OCR 结果：", repr(text.strip()))

pytesseract: 0.3.13
tesseract 版本: 5.5.1
已保存合成测试图 tess_smoke_input.png
OCR 结果： '0128456789,128,458'


In [28]:
# test_sprite_ocr.py
import re, sys
from io import BytesIO
from PIL import Image, ImageOps
import numpy as np
import re, numpy as np
from PIL import Image, ImageOps, ImageFilter
import pytesseract as pt


TESSCFG = (
    "--psm 13 --oem 1 --dpi 400 "
    "-c tessedit_char_whitelist=0123456789, "
    "-c classify_bln_numeric_mode=1 "
    "-c load_system_dawg=0 -c load_freq_dawg=0"
)

def to_pil_L(arr) -> Image.Image:
    """把 numpy 2D 数组安全转成 PIL L(8-bit) 图，兼容 Pillow 新版"""
    arr = np.asarray(arr)
    if arr.dtype != np.uint8:
        arr = arr.astype(np.uint8)
    im = Image.fromarray(arr)        # 不再传 mode
    return im if im.mode == "L" else im.convert("L")      # ← 不再用 mode 参数

def _enhance_red(tile_rgba: Image.Image) -> Image.Image:
    r, g, b, a = tile_rgba.split()
    r = np.asarray(r, dtype=np.float32)
    g = np.asarray(g, dtype=np.float32)
    b = np.asarray(b, dtype=np.float32)

    m = 2 * r - g - b
    m -= m.min()
    m /= (np.ptp(m) + 1e-5)   # numpy 2.x 用 np.ptp
    m *= 255.0
    return to_pil_L(m)        # ← 不再用 mode 参数

def _binarize(img_gray: Image.Image, local=False) -> Image.Image:
    if not local:
        arr = np.asarray(img_gray)
        thr = int(arr.mean())
        bw = img_gray.point(lambda p: 255 if p > thr else 0, mode='1').convert("L")
        return bw
    else:
        arr = np.asarray(img_gray, dtype=np.float32)
        # 这里用 9x9 盒式均值做个简易局部阈值（不依赖 OpenCV）
        k = 9
        pad = k // 2
        # 简易“滑动均值”
        csum = np.cumsum(np.cumsum(arr, axis=0), axis=1)
        def box_sum(x0,y0,x1,y1):
            s = csum[x1,y1]
            if x0>0: s -= csum[x0-1,y1]
            if y0>0: s -= csum[x1,y0-1]
            if x0>0 and y0>0: s += csum[x0-1,y0-1]
            return s
        H, W = arr.shape
        mean = np.zeros_like(arr)
        for i in range(H):
            x0 = max(0, i-pad); x1 = min(H-1, i+pad)
            for j in range(W):
                y0 = max(0, j-pad); y1 = min(W-1, j+pad)
                area = (x1-x0+1)*(y1-y0+1)
                mean[i,j] = box_sum(x0,y0,x1,y1)/area
        bw = (arr > (mean - 5)).astype(np.uint8) * 255
        return to_pil_L(bw)    # ← 不再用 mode 参数

def _thicken(bw: Image.Image, iters=1) -> Image.Image:
    # MinFilter 在黑前景时相当于膨胀（增粗笔画）
    out = bw
    for _ in range(iters):
        out = out.filter(ImageFilter.MinFilter(3))
    return out

def _pad(img: Image.Image, pad=6) -> Image.Image:
    return ImageOps.expand(img, border=pad, fill=255)

def _scale_nearest(img: Image.Image, scale=7) -> Image.Image:
    return img.resize((img.width*scale, img.height*scale), Image.NEAREST)

def _ocr_with_conf(img: Image.Image) -> tuple[str, int]:
    # 用 image_to_data 拿置信度（0-100）
    data = pt.image_to_data(img, config=TESSCFG, output_type=pt.Output.DICT)
    confs = [int(c) for c in data.get("conf", []) if c not in ("-1", "")]
    text  = pt.image_to_string(img, config=TESSCFG)
    ch = re.sub(r'[^0-9,]', '', text).strip()
    conf = max(confs) if confs else (0 if not ch else 60)
    return (ch[:1] if ch else "", conf)

def ocr_digit_tile(tile: Image.Image) -> tuple[str, int, Image.Image]:
    """
    输入：10×16 的字符小块（RGBA/RGB）
    输出：(最佳字符, 置信度, 最佳预处理图)
    """
    tile_rgba = tile.convert("RGBA")

    # 两套配方：A（红通道增强+全局阈值） / B（灰度+局部阈值）
    candidates = []

    # A 路线
    a = _enhance_red(tile_rgba)
    a = _scale_nearest(a, 7)
    a = _binarize(a, local=False)
    a = _thicken(a, iters=1)
    a = _pad(a, 6)
    ch, cf = _ocr_with_conf(a); candidates.append((ch, cf, a))

    # A' 反相再试（个别格有效）
    ainv = ImageOps.invert(a)
    ch2, cf2 = _ocr_with_conf(ainv); candidates.append((ch2, cf2, ainv))

    # B 路线
    b = ImageOps.grayscale(tile_rgba)
    b = _scale_nearest(b, 8)
    b = _binarize(b, local=True)
    b = _thicken(b, iters=1)
    b = _pad(b, 6)
    ch3, cf3 = _ocr_with_conf(b); candidates.append((ch3, cf3, b))

    # 取置信度最高的一个；若都空，则返回空字符
    best = max(candidates, key=lambda x: (x[0] != "", x[1]))
    return best  # (char, conf, image_used)




SPRITE_PATH = "sprite_ok.png"   # ← 改成你的文件名（png/webp/avif/bin 都行）
CELL_W, CELL_H = 10, 16         # ← 你的 CSS 写的是 10x16

# --- 打开图片（含 AVIF 兜底） ---
raw = open(SPRITE_PATH, "rb").read()
def open_image_bytes(raw, content_type_hint=""):
    from PIL import Image
    try:
        img = Image.open(BytesIO(raw)); img.load()
        return img.convert("RGBA")
    except Exception as e1:
        # 尝试 AVIF 插件
        try:
            from pillow_heif import register_heif_opener
            register_heif_opener()
            img = Image.open(BytesIO(raw)); img.load()
            return img.convert("RGBA")
        except Exception:
            try:
                import pillow_avif  # noqa: F401
                img = Image.open(BytesIO(raw)); img.load()
                return img.convert("RGBA")
            except Exception as e2:
                raise RuntimeError("无法打开图片，若为 AVIF 请安装 pillow-heif 或 pillow-avif-plugin") from e2

img = open_image_bytes(raw)
print("雪碧图尺寸:", img.size, "mode:", img.mode)

# --- 切出前 16 个小块（通常前 11 个是 0-9 和 ,） ---
tiles = []
cols = img.width // CELL_W
rows = img.height // CELL_H
for r in range(rows):
    for c in range(cols):
        box = (c*CELL_W, r*CELL_H, (c+1)*CELL_W, (r+1)*CELL_H)
        tiles.append(img.crop(box))
tiles = tiles[:16]


for tile in tiles:
    tile.show()

# --- 预处理 + OCR 每个格 ---
try:
    import pytesseract as pt
except Exception as e:
    print("pytesseract 未安装：", e, "\n请先: pip install pytesseract")
    sys.exit(1)

def prep(tile, scale=6):
    big = tile.resize((tile.width*scale, tile.height*scale), Image.NEAREST)
    gray = ImageOps.grayscale(big)
    arr = np.array(gray)
    thr = int(arr.mean())  # 简单阈值
    bw = gray.point(lambda p: 255 if p > thr else 0, mode='1').convert("L")
    return bw

idx2char = {}
for i, t in enumerate(tiles[:16]):
    ch, conf, img_used = ocr_digit_tile(t)
    print(f"格 {i:02d} -> '{ch}'  conf={conf}")
    # img_used.save(f"debug_tile_{i:02d}.png")  # 如需观察预处理效果

print("\nOCR 识别映射（索引→字符，空串表示没识别到）：")
print(idx2char)
print("\n提示：最终解码用法是 idx = (-offset_x)//10 → 再查 idx2char[idx]")


雪碧图尺寸: (110, 16) mode: RGBA


Error processing line 1 of /Users/syu/miniconda3/envs/YamagotiProjects/lib/python3.10/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "/Users/syu/miniconda3/envs/YamagotiProjects/lib/python3.10/site.py", line 195, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
Error processing line 1 of /Users/syu/miniconda3/envs/YamagotiProjects/lib/python3.10/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "/Users/syu/miniconda3/envs/YamagotiProjects/lib/python3.10/site.py", line 195, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
Error processing line 1 of /Users/syu/miniconda3/envs/YamagotiProjects/lib/python3.10/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "/Users/s

格 00 -> '1'  conf=93
格 01 -> ''  conf=-1
格 02 -> '6'  conf=95
格 03 -> '5'  conf=95
格 04 -> '8'  conf=72
格 05 -> '3'  conf=95
格 06 -> '2'  conf=0
格 07 -> '7'  conf=53
格 08 -> ''  conf=-1
格 09 -> '9'  conf=89
格 10 -> '7'  conf=44

OCR 识别映射（索引→字符，空串表示没识别到）：
{}

提示：最终解码用法是 idx = (-offset_x)//10 → 再查 idx2char[idx]


In [18]:
import re, math, numpy as np
from bs4 import BeautifulSoup

def collect_offsets_x(soup):
    xs = []
    for sp in soup.select('.encrypt-num'):
        st = sp.get('style','')
        m = re.search(r'background-position\s*:\s*(-?\d+)px(?:\s+(-?\d+)px)?', st)
        if not m:
            m = re.search(r'(-?\d+)px', st)
        if m:
            xs.append(int(m.group(1)))
    return xs

def infer_step_from_offsets(xs):
    xs = sorted(set(xs))
    diffs = [abs(xs[i]-xs[i-1]) for i in range(1,len(xs))]
    diffs = [d for d in diffs if d>0]
    if not diffs: return None
    step = diffs[0]
    for d in diffs[1:]:
        step = math.gcd(step, d)
    return step

In [20]:
# 用法：

with requests.Session() as s:
    s.headers.update({
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
        'Accept-Language': 'ja,en;q=0.9,zh-CN;q=0.8',
    })
    # 1) 先进首页拿会话 + CSRF
    r0 = s.get('https://www.kaitorishouten-co.jp/')
    token = extract_csrf_token(r0.text)
    # （可选）再进分类页，部分站点 token 会在分类页覆盖/刷新
    r1 = s.get(LIST_URL)
    token2 = extract_csrf_token(r1.text) or token
    # print('final token =', token2)
    # 2) 带 XHR + Referer + CSRF 访问接口（GET，返回 HTML 片段）
    headers = {
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'https://www.kaitorishouten-co.jp/category/1/709',
        'Accept': '*/*',
    }
    if token2:
        headers['eccube-csrf-token'] = token2
    try:
        r2 = s.get(LIST_URL, headers=headers)
        print('status:', r2.status_code, r2.headers.get('Content-Type'))
    except requests.exceptions.RequestException as e:
        print(e)


soup = BeautifulSoup(r2.content, 'html.parser')
xs = collect_offsets_x(soup)
print("页面X偏移集合:", sorted(set(xs)))
print("由页面推断的步长(GCD):", infer_step_from_offsets(xs))

status: 200 text/html; charset=UTF-8
页面X偏移集合: [-100, -80, -70, -50, -40, -30, -20, -10, 0]
由页面推断的步长(GCD): 10


In [26]:
import cv2, numpy as np
from PIL import Image, ImageOps

def to_gray_uint8(im):
    g = ImageOps.grayscale(im)
    a = np.asarray(g, dtype=np.uint8)
    return a

def build_templates_from_sprite(sprite_img, cell_w=10, cell_h=16, n=11):
    T = []
    for i in range(n):
        box = (i*cell_w, 0, (i+1)*cell_w, cell_h)  # 单行雪碧图
        T.append(to_gray_uint8(sprite_img.crop(box)))
    return T  # list[np.ndarray HxW]

def match_digit(tile_img, templates):
    """返回(最佳模板索引, 得分)；tile_img/模板须同尺寸"""
    q = to_gray_uint8(tile_img)
    best_idx, best_score = -1, -1.0
    for i, tpl in enumerate(templates):
        # 归一化互相关：1.0 越接近越好
        res = cv2.matchTemplate(q, tpl, method=cv2.TM_CCOEFF_NORMED)
        score = float(res[0,0])
        if score > best_score:
            best_score, best_idx = score, i
    return best_idx, best_score

In [29]:
import cv2, numpy as np
from PIL import Image, ImageOps
import PIL

SPRITE_PATH = "sprite_ok.png"   # ← 改成你的文件名（png/webp/avif/bin 都行）
CELL_W, CELL_H = 10, 16         # ← 你的 CSS 写的是 10x16

# --- 打开图片（含 AVIF 兜底） ---
raw = open(SPRITE_PATH, "rb").read()
def open_image_bytes(raw, content_type_hint=""):
    from PIL import Image
    try:
        img = Image.open(BytesIO(raw)); img.load()
        return img.convert("RGBA")
    except Exception as e1:
        # 尝试 AVIF 插件
        try:
            from pillow_heif import register_heif_opener
            register_heif_opener()
            img = Image.open(BytesIO(raw)); img.load()
            return img.convert("RGBA")
        except Exception:
            try:
                import pillow_avif  # noqa: F401
                img = Image.open(BytesIO(raw)); img.load()
                return img.convert("RGBA")
            except Exception as e2:
                raise RuntimeError("无法打开图片，若为 AVIF 请安装 pillow-heif 或 pillow-avif-plugin") from e2

img = open_image_bytes(raw)
print("雪碧图尺寸:", img.size, "mode:", img.mode)

sprite_img = PIL.Image.open("sprite_ok.png").convert("RGBA")
templates = build_templates_from_sprite(sprite_img, 10,16, n=11)
idx, score = match_digit(img, templates)  # idx ∈ [0..10]
# 用 idx → 映射到真实字符（一次性手动把 0..10 标成 '2','5','0',...','）


雪碧图尺寸: (110, 16) mode: RGBA


In [31]:
print(idx, score)

0 1.0


In [ ]:
# -*- coding: utf-8 -*-
"""
模板匹配（非 OCR）方式解码携帯商店雪碧图数字：
- 只下载 分类页 + 雪碧图 两个请求（兜底会多 1 个 big 视图）
- 第一次运行需要对 11 个格手动标注（0-9 和 ,），按雪碧图 MD5 缓存
- 下次遇到 新雪碧图（顺序打乱），用 pHash + 相关系数 自动匹配，无需再次标注
"""

import os, json, re, hashlib, sys
from io import BytesIO
from urllib.parse import urljoin
import requests
import numpy as np
from bs4 import BeautifulSoup
from PIL import Image, ImageOps, ImageDraw
import imagehash
import csv, re
# ========= 配置 =========
_HERE = os.path.dirname(os.path.abspath("__file__")) if "__file__" not in dir() else os.path.dirname(os.path.abspath(__file__))
BASE = "https://www.kaitorishouten-co.jp"
CID  = "709"  # ← 改成你的分类ID（例如 iPhone 17 Pro Max = 711）
CAT_URL = f"{BASE}/category/1/{CID}"
HEADERS = {"User-Agent": "Mozilla/5.0", "Accept-Language": "ja,en;q=0.9,zh-CN;q=0.8"}
CACHE_SPRITE2IDXCHAR = os.path.join(_HERE, "sprite_idxchar_by_md5.json")   # 雪碧图MD5 → {idx:char}
TEMPLATE_DB = os.path.join(_HERE, "digit_templates.json")                  # 全局模板库：phash(hex) → {'ch': '0', 'w':10,'h':16}
TEMPLATE_IMG_DIR = os.path.join(_HERE, "digit_templates")                  # 可视化保存每个模板的小图
os.makedirs(TEMPLATE_IMG_DIR, exist_ok=True)

LIST_URL = f"{BASE}/category/1/{CID}"
# =======================

def md5(b: bytes) -> str:
    return hashlib.md5(b).hexdigest()



def extract_sprite_info(soup):
    # 从 <style> 找到 .encrypt-num 的 background-image 与宽高
    css = "\n".join((s.string or s.text or "") for s in soup.find_all("style"))
    m_url = re.search(r'\.encrypt-num[^}]*background-image\s*:\s*url\((["\']?)([^)\'"]+)\1\)', css)
    if not m_url:
        return None, 10, 16
    sprite_url = urljoin(BASE, m_url.group(2).strip())
    m_w = re.search(r'\.encrypt-num[^}]*width\s*:\s*(\d+)px', css)
    m_h = re.search(r'\.encrypt-num[^}]*height\s*:\s*(\d+)px', css)
    cell_w = int(m_w.group(1)) if m_w else 10
    cell_h = int(m_h.group(1)) if m_h else 16
    return sprite_url, cell_w, cell_h

def open_image_bytes(raw, content_type_hint=""):
    # 打开 PNG/WEBP/AVIF（若为 AVIF 需 pillow-heif 或 pillow-avif-plugin）
    try:
        img = Image.open(BytesIO(raw)); img.load()
        return img.convert("RGBA")
    except Exception:
        # AVIF 兜底
        try:
            from pillow_heif import register_heif_opener
            register_heif_opener()
            img = Image.open(BytesIO(raw)); img.load()
            return img.convert("RGBA")
        except Exception:
            try:
                import pillow_avif  # noqa
                img = Image.open(BytesIO(raw)); img.load()
                return img.convert("RGBA")
            except Exception as e2:
                raise RuntimeError("无法打开图片，若为 AVIF 请安装 pillow-heif 或 pillow-avif-plugin") from e2

def download_sprite(sess, list_url, sprite_url):
    r = sess.get(sprite_url, headers={
        **HEADERS,
        "Referer": list_url,
        "Accept": "image/webp,image/png,*/*;q=0.8",  # 尽量避免 AVIF
    }, timeout=30)
    r.raise_for_status()
    raw = r.content
    ct = r.headers.get("Content-Type","").lower()
    if (ct.startswith("text/html") or not ct) and (raw[:64].lstrip().startswith(b"<!") or b"<html" in raw[:256].lower()):
        open("sprite_debug.html","wb").write(raw)
        raise RuntimeError("拿到 HTML（反盗链/旧 token）。已保存 sprite_debug.html")
    return open_image_bytes(raw, ct), raw, ct

def cut_row_tiles(sprite_img: Image.Image, cell_w: int, cell_h: int) -> list[Image.Image]:
    """假设单行雪碧图；如果是多行，可自行扩展循环 y 行"""
    cols = sprite_img.width // cell_w
    tiles = []
    for c in range(cols):
        x0 = c*cell_w
        tiles.append(sprite_img.crop((x0, 0, x0+cell_w, cell_h)))
    return tiles

def tiles_phash(tiles):
    """对每个小格计算 pHash（16x16），返回 list[str(hex)]"""
    hashes = []
    for t in tiles:
        # 先转灰度；pHash 对轻微压缩/噪声鲁棒
        h = imagehash.phash(ImageOps.grayscale(t), hash_size=8)  # 64-bit
        hashes.append(h.__str__())
    return hashes

def load_json(path, default):
    if os.path.exists(path):
        with open(path,"r",encoding="utf-8") as f:
            return json.load(f)
    return default

def save_json(path, data):
    with open(path,"w",encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def show_and_label_once(tiles, cell_w, cell_h):
    """首次遇到某张雪碧图时，输出预览让你一次性标注 11 个字符"""
    grid = Image.new("RGB", (len(tiles)*cell_w, cell_h), "white")
    draw = ImageDraw.Draw(grid)
    x = 0
    for i, t in enumerate(tiles):
        grid.paste(t.convert("RGB"), (x,0))
        draw.text((x+1,1), str(i), fill=(255,0,0))
        x += cell_w
    big = grid.resize((grid.width*8, grid.height*8), Image.NEAREST)
    big.save("sprite_tiles_preview.png")
    print("请打开 sprite_tiles_preview.png，对照从左到右输入每格字符（仅 0-9 与 ,）")
    idx2char = {}
    for i in range(len(tiles)):
        while True:
            ch = input(f"第 {i} 格 = ").strip()
            if ch in "0123456789,":
                idx2char[str(i)] = ch
                break
            print("只能输入 0-9 或 ,")
    return idx2char

def build_or_update_template_db(tiles, labels, db_path=TEMPLATE_DB):
    """把本次标注的每个 tile 作为"字符模板"存进全局库（pHash → char）"""
    db = load_json(db_path, {})
    for i, t in enumerate(tiles):
        ch = labels[str(i)]
        h = imagehash.phash(ImageOps.grayscale(t), hash_size=8).__str__()
        db[h] = {"ch": ch, "w": t.width, "h": t.height}
        # 也存一份方便可视化
        t.save(os.path.join(TEMPLATE_IMG_DIR, f"{ch}_{h}.png"))
    save_json(db_path, db)
    return db

def hamming(a_hex: str, b_hex: str) -> int:
    return bin(int(a_hex,16) ^ int(b_hex,16)).count("1")

def map_idx_to_char_by_templates(tiles, db, ham_thr=6):
    """
    对当前雪碧图的小格，用模板库做匹配，得到 idx→char。
    先用 pHash 最近邻（汉明距离<=阈值），否则再用相关系数打分。
    """
    idx2char = {}
    # 预备模板数组
    tpl_imgs = []
    tpl_chars = []
    for h, meta in db.items():
        path = None
        # 尝试从磁盘读模板小图；若不存在，仅凭 phash 匹配
        for f in os.listdir(TEMPLATE_IMG_DIR):
            if f.endswith(f"{h}.png"):
                path = os.path.join(TEMPLATE_IMG_DIR, f)
                break
        if path and os.path.exists(path):
            tpl_imgs.append(Image.open(path).convert("L"))
            tpl_chars.append(meta["ch"])
        else:
            tpl_imgs.append(None)
            tpl_chars.append(meta["ch"])
    tpl_hashes = list(db.keys())

    def ncc(a: Image.Image, b: Image.Image) -> float:
        # 归一化互相关（尺寸必须一致）
        A = np.asarray(ImageOps.grayscale(a), dtype=np.float32)
        B = np.asarray(ImageOps.grayscale(b.resize(a.size, Image.NEAREST)), dtype=np.float32)
        A = (A - A.mean()) / (A.std() + 1e-6)
        B = (B - B.mean()) / (B.std() + 1e-6)
        return float((A*B).mean())

    for i, t in enumerate(tiles):
        h = imagehash.phash(ImageOps.grayscale(t), hash_size=8).__str__()
        # 第一轮：pHash 最近邻
        best_j, best_d = None, 999
        for j, th in enumerate(tpl_hashes):
            d = hamming(h, th)
            if d < best_d:
                best_d, best_j = d, j
        if best_d <= ham_thr:
            idx2char[i] = tpl_chars[best_j]
            continue
        # 第二轮：NCC 与所有有图模板比一遍
        best_score, best_char = -2.0, None
        for j, tpl in enumerate(tpl_imgs):
            if tpl is None:  # 没有模板图，跳过
                continue
            score = ncc(t, tpl)
            if score > best_score:
                best_score, best_char = score, tpl_chars[j]
        if best_char is not None:
            idx2char[i] = best_char
        else:
            idx2char[i] = "?"  # 仍然未知，稍后可手动补

    return idx2char

def decode_row_price(tr, cell_w, idx2char):
    out = []
    for sp in tr.select("span.encrypt-num"):
        st = sp.get("style","")
        m = re.search(r'(-?\d+)px', st)
        if not m: out.append("?"); continue
        x = int(m.group(1))
        idx = (-x) // cell_w
        out.append(idx2char.get(idx, "?"))
    return "".join(out)

def main(CID):
    LIST_URL = f"{BASE}/category/1/{CID}"
    CAT_URL = f"{BASE}/category/1/{CID}"
    with requests.Session() as s:
        s.headers.update({
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
        'Accept-Language': 'ja,en;q=0.9,zh-CN;q=0.8',
        })
        # 1) 先进首页拿会话 + CSRF
        r0 = s.get('https://www.kaitorishouten-co.jp/')
        token = extract_csrf_token(r0.text)
        # （可选）再进分类页，部分站点 token 会在分类页覆盖/刷新
        r1 = s.get(LIST_URL)
        token2 = extract_csrf_token(r1.text) or token
        # print('final token =', token2)
        # 2) 带 XHR + Referer + CSRF 访问接口（GET，返回 HTML 片段）
        headers = {
            'X-Requested-With': 'XMLHttpRequest',
            'Referer': 'https://www.kaitorishouten-co.jp/category/1/709',
            "Accept": "image/webp,image/png,*/*;q=0.8",
        }
        if token2:
            headers['eccube-csrf-token'] = token2
        try:
            r2 = s.get(LIST_URL, headers=headers)
            # print('status:', r2.status_code, r2.headers.get('Content-Type'))
        except requests.exceptions.RequestException as e:
            print(e)

        r2.raise_for_status()
        r2.headers.get("Content-Type","").startswith("image/")
        soup = BeautifulSoup(r2.content, "html.parser")

        # 2) 找雪碧图 + 单格尺寸
        sprite_url, cell_w, cell_h = extract_sprite_info(soup)
        if not sprite_url:
            print("未找到 .encrypt-num 的 background-image；检查页面。"); return
        # print("Sprite URL:", sprite_url, "| cell:", cell_w, "x", cell_h)

        # 3) 下载雪碧图
        sprite_img, sprite_bytes, ct = download_sprite(s, CAT_URL, sprite_url)
        sprite_md5 = md5(sprite_bytes)
        # print("Sprite MD5:", sprite_md5, "|", ct)

        # 4) 切出模板格（默认单行 11~12 格）
        tiles = cut_row_tiles(sprite_img, cell_w, cell_h)
        # 只取前 16 格（通常 11 格就够）
        tiles = tiles[:16]

        # 5) 如果这张雪碧图之前标注过，直接用；否则：用模板库自动匹配 → 仍缺就手动标注
        all_maps = load_json(CACHE_SPRITE2IDXCHAR, {})
        idx2char = all_maps.get(sprite_md5)

        if not idx2char:
            # 尝试用历史模板自动匹配
            tpl_db = load_json(TEMPLATE_DB, {})
            if tpl_db:
                auto_map = map_idx_to_char_by_templates(tiles, tpl_db, ham_thr=6)
                # print("自动匹配得到（可能不全）：", auto_map)
                if len([v for v in auto_map.values() if v != "?"]) >= 10:
                    idx2char = {str(i): ch for i, ch in auto_map.items()}
            # 若仍然不全，走一次人工标注（只此一次）
            if not idx2char or "?" in idx2char.values():
                labels = show_and_label_once(tiles, cell_w, cell_h)  # {'0':'2','1':'4',...}
                # 把标注写入全局模板库
                build_or_update_template_db(tiles, labels, db_path=TEMPLATE_DB)
                idx2char = labels

            # 记到"按雪碧图MD5"的缓存
            all_maps[sprite_md5] = idx2char
            save_json(CACHE_SPRITE2IDXCHAR, all_maps)
        # print("本次 idx→char 映射：", idx2char)

        # 6) 解码整页价格并打印
        # print("\n=== 解码结果 ===")
        rows = []
        for tr in soup.select("tr.price_list_item"):
            # 价格（仍用你已有的函数和映射）
            price = decode_row_price(tr, cell_w, {int(k): v for k, v in idx2char.items()})
            price = price.replace(',', '')

            # 提取 JAN：优先从两个紧邻的 span.product-code-default 里拿
            jan = None
            spans = tr.select('span.product-code-default')
            for i, sp in enumerate(spans):
                if sp.get_text(strip=True).upper().startswith('JAN'):
                    if i + 1 < len(spans):
                        jan = spans[i + 1].get_text(strip=True)
                    break
            # 兜底：从整行文本里正则匹配
            if not jan:
                m = re.search(r'JAN:\s*([0-9]{8,14})', tr.get_text(' ', strip=True))
                if m:
                    jan = m.group(1)

            if jan:
                rows.append({'JAN': jan, 'price': price})

        with open('shop1.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=['JAN', 'price'])
            writer.writeheader()
            writer.writerows(rows)

        print('已写入 shop1.csv')

In [54]:
CID_LIST = ["711","710","709","708"]
for cid in CID_LIST:
    main(cid)

已写入 shop1.csv
已写入 shop1.csv
已写入 shop1.csv
已写入 shop1.csv
